### Quickstart for TARDIS ###

Every simulation run requires the atomic database (for more info refer to [atomic data](../using/components/atomic/atomic_data.rst) ) and a configuration file (more info at [configuration](../using/components/configuration/index.rst) ).
You can obtain a copy of the atomic database from the
(https://github.com/tardis-sn/tardis-refdata) repository
(`atom_data` subfolder). We recommended to use the
`kurucz_cd23_chianti_H_He.h5` dataset (which is auto-downloaded in the second cell already). The configuration file for this quickstart is `tardis_example.yml`, which can be downloaded [here](https://raw.githubusercontent.com/tardis-sn/tardis/master/docs/using/components/models/examples/tardis_example.yml)), though this file is auto-downloaded in the third cell.

After the [installation](../installation.rst), start a Jupyter server executing `jupyter notebook` on the command line in a directory that contains this quickstart.

In [ ]:
from tardis import run_tardis
from tardis.io.atom_data.util import download_atom_data

#### Downloading the atomic data ####

In [ ]:
# the data is automatically downloaded
download_atom_data('kurucz_cd23_chianti_H_He')

#### Downloading the example file ####

In [ ]:
!curl -O https://raw.githubusercontent.com/tardis-sn/tardis/master/docs/tardis_example.yml

#### Running the simulation (long output) ####

In [ ]:
#TARDIS now uses the data in the data repo
sim = run_tardis('tardis_example.yml')

#### Plotting the Spectrum ####

In [ ]:
%pylab inline

spectrum = sim.runner.spectrum
spectrum_virtual = sim.runner.spectrum_virtual
spectrum_integrated = sim.runner.spectrum_integrated

figure(figsize=(10,6))
spectrum.plot(label='normal packets')
spectrum_virtual.plot(label='virtual packets')
spectrum_integrated.plot(label='formal integral')
xlabel('Wavelength [$\AA$]')
ylabel('Luminosity [erg/s/$\AA$]')
legend()
xlim(3000, 9000)